In [173]:
import datetime
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA, GOOG
from bokeh.plotting import figure, output_file, save
from matplotlib import category
from TrainAndTest import _exponential_smooth, _produce_movement_indicators

from bybitapi import fetch_bybit_data_v5
from config import INTERVAL
import pandas as pd
import backtesting



class SmaCross(Strategy):
    n1 = 10
    n2 = 20

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):
        if True:
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()



class Train():
    trainingdata = []
        
    def get_data(self):
        end_date = datetime.datetime.now()
        start_date = end_date -datetime.timedelta(2)
        #fetch the data
        self.trainingdata = fetch_bybit_data_v5(True,start_date,end_date,"BTCUSDT",INTERVAL,'spot')
        #smooth the data
        self.trainingdata = _exponential_smooth(self.trainingdata,0.65)
        #produce indicators
        self.trainingdata = _produce_movement_indicators(self.trainingdata)
        return self.trainingdata


In [174]:
GOOG.tail()

,Open,High,Low,Close,Volume
2013-02-25,802.3,808.41,790.49,790.77,2303900
2013-02-26,795.0,795.95,784.40,790.13,2202500
2013-02-27,794.8,804.75,791.11,799.78,2026100
2013-02-28,801.1,806.99,801.03,801.20,2265800
2013-03-01,797.8,807.14,796.15,806.19,2175400


In [175]:
train = Train()
DATA = train.get_data()

there are  2 total days in the range.
total_intervals: 576.0
intervals: [[1688460228.031082, 1688520228.031082], [1688520228.031082, 1688580228.031082], [1688580228.031082, 1688640228.031082]]
Fetching historical data from 2023-07-04 08:43:48.031082 to 2023-07-05 01:23:48.031082
Fetching historical data from 2023-07-05 01:23:48.031082 to 2023-07-05 18:03:48.031082
Fetching historical data from 2023-07-05 18:03:48.031082 to 2023-07-06 10:43:48.031082
returning new
                             open_time      open      high       low  \
timestamp                                                              
2023-07-04 08:45:28.960  1688460300000     30951  30954.03  30923.44   
2023-07-04 08:49:51.104  1688460600000  30923.44  30942.99     30916   
2023-07-04 08:54:13.248  1688460900000  30935.43  30940.02     30910   
2023-07-04 09:00:46.464  1688461200000  30927.99     30934     30900   
2023-07-04 09:05:08.608  1688461500000     30900  30948.33  30871.65   
...                         

/workspace/bybitapi.py:99: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_new['timestamp'] = pd.to_datetime(df_new['open_time'], unit='ms')


In [190]:

df = DATA.loc[:, ['open', 'high', 'low', 'close', 'volume']]
df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
df.index = df.index.strftime('%Y-%m-%d %H:%M')
df.index = pd.to_datetime(df.index)
# df= df.tail(5)

# DATA= train.get_data().tail()
# df = df.apply(pd.to_numeric)
# df['Volume'] = df['Volume'].astype('float64')


In [193]:

backtesting.set_bokeh_output(notebook=False)
bt = Backtest(df, SmaCross,
              cash=100000, commission=.002,
              exclusive_orders=True)
output = bt.run()
bt.plot()
print(output)
    


/usr/local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p4989', ...)